# CS 1656 – Introduction to Data Science 

## Instructor: Alexandros Labrinidis
## Teaching Assistants: Evangelos Karageorgos, Xiaoting Li, Zi Han Ding
### Additional credits: Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation : SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [ ]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [ ]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 270 days ago)
# end_date = datetime.now()
# start_date = end_date - timedelta(days=270)

# Get two date endpoints
start_date = date(2021, 9, 1)
end_date = date(2022, 6, 1)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}'""".format(resource_id, start_str, end_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")
print(df.columns)
df.head()


Details of useful dataset attributes are below. ((Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)

* __facility_name__: the name of the facility
* __description__: Facility category
* __description_new__: The name of the potential violation
* __inspect_dt__: Date/time of the inspection
* __rating__: The result of the inspection ('V' for violation, other for non-violation)
* The health risk of a potential violation
 * __low__: low risk
 * __medium__: medium risk
 * __high__: high risk
* The address of the facility
 * __city__: The city
 * __state__: The state
 * __street__: The street
 * __num__: The street number
 * __zip__: The zip code

## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the time span (violation description[violation]).__

In [ ]:
query = """
SELECT DISTINCT "description_new" as violation
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' """.format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category (facility name[facility], number of violations[count]). NOTE: a facility has a violation if the inspection rating has the value 'V'.__

In [ ]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

# Tasks

__Tasks 1 to 4 must be implemented in Task.py.__

__For all tasks, we want the results in Pittsburgh, over the specified time span, and all queries are about violations (rating is 'V').__

__As the API returns the results as a list of dictionaries, the order of the query columns is irrelevant.__


__T1) Find the top 20 facilities that start with 'Pitt' and have the highest counts of violations _(facility name[facility], number of violations[count])_.__

__T2) Find the top 18 restaurants with the maximum number of violations _(facility name[facility], number of violations[count])_. Include all results in case of a tie (For example, if the 18th top restaurant has 10 violations, incude all other restaurants with 10 violations). HINT: You will need an extra query to get the tie-breaker value.__

__T3) Find the facilities that start with 'Pitt' and have violations over the time span _(violation description[violation], number of facilities[count], facility names[facilities])_. The _facilities_ field must be a concatenation of all facility names, in alphabetical order, seperated by a comma and a space (', ').__

Now lets look at all facilities that contain word 'Pitt'.

__T4) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included _(facility name[facility], violation description[violation], high[high], medium[medium], low[low])_. HINT: consider _all_ edge cases for identifying 'Pitt' as a seperate word.__